In [4]:
import cherrypy
import json
import redis

#redis_client = redis.Redis(
  #username='default',
  #host='redis-19264.c301.ap-south-1-1.ec2.cloud.redislabs.com',
  #port=19264,
  #password='6HVuWRH8CSONqykRKS9Y2esqvv6o4II7'
  #)
redis_client = redis.Redis(
  username='default',
  password='g49VWZnOYYr1TVC7CqwyDOY48wa6BHff',
  host='redis-14675.c250.eu-central-1-1.ec2.cloud.redislabs.com',
  port=14675
  )

# endpoint /devices
class Devices(object):
    exposed = True

    def GET(self, *path, **query):
        devices = [key.decode().split(':')[0] for key in redis_client.keys('*:power')]
        return json.dumps({'mac_addresses' : devices})

# endpoint /device/{mac_address}
class BatteryStatus(object):
    exposed = True
    
    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        if 'from' not in query:
            raise cherrypy.HTTPError(400, 'Bad Request: missing start time')
        if 'to' not in query:
            raise cherrypy.HTTPError(400, 'Bad Request: missing end time')
        mac_address = path[0]
        start, end = int(query['from']), int(query['to'])
        devices = [key.decode().split(':')[0] for key in redis_client.keys('*:power')]
        if mac_address not in devices:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')
        battery_ts = redis_client.ts().range(mac_address + ':battery', start, end)
        power_ts = redis_client.ts().range(mac_address + ':power', start, end)
        if battery_ts and power_ts: 
            timestamps, battery_levels = zip(*battery_ts)
            _, power = zip(*power_ts)
        else:
            timestamps, battery_levels, power = [], [], []
        return json.dumps({
            'mac_address': mac_address,
            'timestamps': timestamps,
            'battery_levels': battery_levels,
            'power_plugged': power
        })
    
    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        mac_address = path[0]
        devices = [key.decode().split(':')[0] for key in redis_client.keys('*:power')]
        if mac_address not in devices:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')
        redis_client.delete(mac_address + ':battery')
        redis_client.delete(mac_address + ':power')
        return 'OK: Everything worked as expected.'
        

conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
cherrypy.tree.mount(Devices(), '/devices', conf)
cherrypy.tree.mount(BatteryStatus(), '/device', conf)
cherrypy.config.update({'server.socket_host': '0.0.0.0'})
cherrypy.config.update({'server.socket_port': 8080})
cherrypy.engine.start()
cherrypy.engine.block()

[06/Jan/2023:15:19:06] ENGINE Bus STARTING
[06/Jan/2023:15:19:06] ENGINE Started monitor thread 'Autoreloader'.
[06/Jan/2023:15:19:07] ENGINE Serving on http://0.0.0.0:8080
[06/Jan/2023:15:19:07] ENGINE Bus STARTED


127.0.0.1 - - [06/Jan/2023:15:19:10] "GET /devices HTTP/1.1" 200 37 "" "python-requests/2.28.1"
127.0.0.1 - - [06/Jan/2023:15:19:10] "GET /device/0xc71f33a20b34?from=1672928350175&to=1673014750175 HTTP/1.1" 200 825 "" "python-requests/2.28.1"
127.0.0.1 - - [06/Jan/2023:15:19:10] "DELETE /device/0xc71f33a20b34 HTTP/1.1" 200 34 "" "python-requests/2.28.1"
127.0.0.1 - - [06/Jan/2023:15:19:14] "GET /devices HTTP/1.1" 200 37 "" "python-requests/2.28.1"
127.0.0.1 - - [06/Jan/2023:15:19:15] "GET /device/0xc71f33a20b34?from=1672928354994&to=1673014754994 HTTP/1.1" 200 283 "" "python-requests/2.28.1"
127.0.0.1 - - [06/Jan/2023:15:19:15] "DELETE /device/0xc71f33a20b34 HTTP/1.1" 200 34 "" "python-requests/2.28.1"


[06/Jan/2023:15:19:25] ENGINE Keyboard Interrupt: shutting down bus
[06/Jan/2023:15:19:25] ENGINE Bus STOPPING
[06/Jan/2023:15:19:25] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('0.0.0.0', 8080)) shut down
[06/Jan/2023:15:19:25] ENGINE Stopped thread 'Autoreloader'.
[06/Jan/2023:15:19:25] ENGINE Bus STOPPED
[06/Jan/2023:15:19:25] ENGINE Bus EXITING
[06/Jan/2023:15:19:25] ENGINE Bus EXITED
[06/Jan/2023:15:19:25] ENGINE Waiting for child threads to terminate...
